In [1]:
%matplotlib inline

A few basic libraries that we'll need for the initial exercises:

In [2]:
from __future__ import division,print_function

import os
# import json
from glob import glob
import numpy as np
import pandas as pd
#np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import xgboost as xgb

# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder


/home/adam/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Set up kaggle config for this competition (after agreeing to rules).

In [3]:
#!kg config -g -u `username` -p `password` 

##dogs-vs-cats-redux-kernels-edition
##state-farm-distracted-driver-detection
##titanic
!kg config -c 'titanic'

### Data set-up

In [4]:
# Read names of training images from .csv file
DATA_HOME_DIR = '/home/adam/kaggle/data/titanic/'
%cd $DATA_HOME_DIR

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

other_data = pd.read_csv('gender_submission.csv')

/home/adam/kaggle/data/titanic


In [5]:
## kaggle.com/datacanary/xgboost-example-python/code
## 'Impute missing values using the median for numeric columns and the most common value for string columns.'
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch']
nonnumeric_columns = ['Sex']

## 'Join the features from train and test together before imputing missing values,
## in case their distribution is slightly different'
big_X = train_data[feature_columns_to_use].append(test_data[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

## 'XGBoost doesn't (yet) handle categorical features automatically, so we need to change
## them to columns of integer values.
## See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
## details and options'
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

## 'Prepare the inputs for the model'
train_X = big_X_imputed[0:train_data.shape[0]].as_matrix()
test_X = big_X_imputed[train_data.shape[0]::].as_matrix()
train_y = train_data['Survived']

### Fit model

In [6]:
## 'You can experiment with many other options here, using the same .fit() and .predict()
## methods; see http://scikit-learn.org
## This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost'
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = gbm.predict(test_X)

### Save predictions

In [7]:
## 'Kaggle needs the submission to have a certain format;
## see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
## for an example of what it's supposed to look like.'
submission = pd.DataFrame({ 'PassengerId': test_data['PassengerId'],
                            'Survived': predictions })
submission_fnm = DATA_HOME_DIR+'submission.csv'
submission.to_csv(submission_fnm, index=False)

### Submit

In [8]:
print('submitting file to kaggle: '+submission_fnm)
! kg submit $submission_fnm

submitting file to kaggle: /home/adam/kaggle/data/titanic/submission.csv
list index out of range
